In [28]:
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2

In [29]:
df = pd.read_excel('Modelos_defunciones_meses.xlsx', 'nov_dic_ene', dtype={'CVEGEO': str})

df

,CVEGEO,nom_ent,nom_mun,pobtot_1,P_60YMAS,def_nov,def_dic,def_ene,score_nov,ventil_nov,...,prob_dic,p_dic+(p_dic-p_nov),pronostico ene,Unnamed: 16,Unnamed: 17,Etiquetas de fila,score_nov.1,prob_nov.1,Score_dic.1,prob_dic.1
0,1001,Aguascalientes,Aguascalientes,948990,102987,206,171,125,17.589041,D20,...,0.001530,0.000839,86.454470,NaN,NaN,D01,-58.929446,0.000055,-64.487428,0.000082
1,1002,Aguascalientes,Asientos,51536,4697,3,4,3,-12.970989,D07,...,0.000573,0.000686,3.223119,NaN,NaN,D02,-41.252814,0.000139,-44.827027,0.000133
2,1003,Aguascalientes,Calvillo,58250,7829,5,2,3,-2.860330,D09,...,0.000825,0.001104,8.646370,NaN,NaN,D03,-33.942486,0.000184,-36.356351,0.000222
3,1004,Aguascalientes,Cosío,17000,1613,5,0,0,-18.461754,D06,...,0.000408,0.000469,0.756107,NaN,NaN,D04,-27.367738,0.000230,-28.769154,0.000279
4,1005,Aguascalientes,Jesús María,129929,9815,7,6,4,5.259145,D12,...,0.001030,0.001472,14.445284,NaN,NaN,D05,-21.745225,0.000309,-21.984820,0.000408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446,2126,4,1,0,-20.198300,D05,...,0.000408,0.000508,1.079783,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465,32055,Zacatecas,Villanueva,31558,5638,16,7,2,-9.515222,D08,...,0.000573,0.000697,3.928006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,32056,Zacatecas,Zacatecas,149607,18847,68,47,39,14.458408,D19,...,0.001456,0.001845,34.771871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,32057,Zacatecas,Trancoso,20455,1785,6,3,2,-18.784424,D06,...,0.000488,0.000627,1.119629,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df = df[['CVEGEO', 'nom_ent', 'nom_mun', 'pobtot_1']]
df

,CVEGEO,nom_ent,nom_mun,pobtot_1
0,1001,Aguascalientes,Aguascalientes,948990
1,1002,Aguascalientes,Asientos,51536
2,1003,Aguascalientes,Calvillo,58250
3,1004,Aguascalientes,Cosío,17000
4,1005,Aguascalientes,Jesús María,129929
...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446
2465,32055,Zacatecas,Villanueva,31558
2466,32056,Zacatecas,Zacatecas,149607
2467,32057,Zacatecas,Trancoso,20455


In [31]:
df['CVEGEO'] = df['CVEGEO'].apply(lambda x: x.rjust(5, '0'))

df['gridid_state'] = df['CVEGEO'].str.slice(0, 2)
#list(df['CVEGEO'].unique())

df

/home/pedro/anaconda3/envs/geo-environment/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/pedro/anaconda3/envs/geo-environment/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CVEGEO,nom_ent,nom_mun,pobtot_1,gridid_state
0,01001,Aguascalientes,Aguascalientes,948990,01
1,01002,Aguascalientes,Asientos,51536,01
2,01003,Aguascalientes,Calvillo,58250,01
3,01004,Aguascalientes,Cosío,17000,01
4,01005,Aguascalientes,Jesús María,129929,01
...,...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446,32
2465,32055,Zacatecas,Villanueva,31558,32
2466,32056,Zacatecas,Zacatecas,149607,32
2467,32057,Zacatecas,Trancoso,20455,32


In [32]:
df = df.rename(columns={'CVEGEO': 'gridid_mun'})
df

,gridid_mun,nom_ent,nom_mun,pobtot_1,gridid_state
0,01001,Aguascalientes,Aguascalientes,948990,01
1,01002,Aguascalientes,Asientos,51536,01
2,01003,Aguascalientes,Calvillo,58250,01
3,01004,Aguascalientes,Cosío,17000,01
4,01005,Aguascalientes,Jesús María,129929,01
...,...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446,32
2465,32055,Zacatecas,Villanueva,31558,32
2466,32056,Zacatecas,Zacatecas,149607,32
2467,32057,Zacatecas,Trancoso,20455,32


In [36]:
df_states = df.groupby(['gridid_state', 'nom_ent']).sum()
df_states = df_states.reset_index()
df_states = df_states.rename(columns={'nom_ent': 'name', 'pobtot_1': 'pobtot'})

df_states

,gridid_state,name,pobtot
0,01,Aguascalientes,1425607
1,02,Baja California,3769020
2,03,Baja California Sur,798447
3,04,Campeche,928363
4,05,Coahuila de Zaragoza,3146771
5,06,Colima,731391
6,07,Chiapas,5543828
7,08,Chihuahua,3741869
8,09,Ciudad de México,9209944
9,10,Durango,1832650


In [33]:
db_name = os.environ['DBNAME']
db_user = os.environ['DBUSER']
db_port = os.environ['DBPORT']
db_host = os.environ['DBHOST']
db_pass = os.environ['DBPASS']

In [34]:
str_conn = 'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}'.format(db_name=db_name,
                                                                                           db_user=db_user,
                                                                                           db_port=db_port,
                                                                                           db_host=db_host,
                                                                                           db_pass=db_pass)
engine = create_engine(str_conn)

In [37]:
df_states.to_sql('mesh_state', con=engine, if_exists='append', index=False)

In [38]:
df_states = pd.read_sql('SELECT id, gridid_state FROM mesh_state', con=engine)

df_states

,id,gridid_state
0,1,01
1,2,02
2,3,03
3,4,04
4,5,05
5,6,06
6,7,07
7,8,08
8,9,09
9,10,10


In [39]:
df_muns = df.rename(columns={'nom_ent': 'state_name', 'nom_mun': 'name'})
df_muns = df_muns.merge(df_states, on='gridid_state')
df_muns = df_muns.rename(columns={'id': 'state_id', 'pobtot_1': 'pobtot'})
df_muns

,gridid_mun,state_name,name,pobtot,gridid_state,state_id
0,01001,Aguascalientes,Aguascalientes,948990,01,1
1,01002,Aguascalientes,Asientos,51536,01,1
2,01003,Aguascalientes,Calvillo,58250,01,1
3,01004,Aguascalientes,Cosío,17000,01,1
4,01005,Aguascalientes,Jesús María,129929,01,1
...,...,...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446,32,32
2465,32055,Zacatecas,Villanueva,31558,32,32
2466,32056,Zacatecas,Zacatecas,149607,32,32
2467,32057,Zacatecas,Trancoso,20455,32,32


In [40]:
df_muns.to_sql('mesh_mun', con=engine, if_exists='append', index=False)